# Distraction Driver Detection Project
## Project Aim: To predict the activity of the driver from the image
This document is a test notebook to test and implement the codes for the Distracted Driver detection project.

This is some plain text that forms a paragraph.
Add emphasis via **bold** and __bold__, or *italic* and _italic_.

Paragraphs must be separated by an empty line.

* Sometimes we want to include lists.
 * Which can be indented.

1. Lists can also be numbered.
2. For ordered lists.

[It is possible to include hyperlinks](https://www.example.com)

Inline code uses single backticks: `foo()`, and code blocks use triple backticks:

```
bar()
```

Or can be intented by 4 spaces:

    foo()


## Step 0: Importing the datasets

We start by importing the images from the datasets. The datasets are too large for the memory to load all the data simultaneously. So we start by loading the file names and the labels. We also split the files to train, test and validation sets here, as the actual test set peovided to us has no labels. So we are sticking to validate and test our models with the train set itself. 

In [1]:
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import numpy as np
from glob import glob

#defining a function to load the dataset
def load_dataset(path):
    data = load_files(path)
    driver_images = np.array(data['filenames'])
    driver_activities = np_utils.to_categorical(np.array(data['target']))
    return driver_images, driver_activities

#loading the datasets
#change the directory in the github as well
images, targets = load_dataset('../input/state-farm-distracted-driver-detection/train/')

#splitting data to train, test and validation datasets
images_train, images_rest, targets_train, targets_rest = train_test_split( images, targets, train_size=0.8, random_state=42)
images_val, images_test, targets_val, targets_test = train_test_split( images_rest, targets_rest, train_size=0.5, random_state=42)


#printing the dataset statistics
print('There are %d total number of driver images' % len(images))


print('There are %d number of train images' % len(images_train))
print('There are %d number of validation images' % len(images_val))
print('There are %d number of test images' % len(images_test))


import os
print(os.listdir("../input/state-farm-distracted-driver-detection/train"))


/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


There are 22424 total number of driver images
There are 17939 number of train images
There are 2242 number of validation images
There are 2243 number of test images
['c9', 'c0', 'c8', 'c1', '.DS_Store', 'c7', 'c5', 'c6', 'c4', 'c2', 'c3']


/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


## Step 2: Data Generator

We found our the RAM we have on this computer/kernel is not large enough to run this algorithm for this large dataset. Hence, we train the model in batches. We define a new class  inherited from the Sequence class in keras and modify it to our requirements. The batch size is set to 32 images by default, which we will increase depending on other factors. For preprocessing, we normalize the pixels in the images. Normalization and image size setting is done in the data generator class itself to reduce the need for preprocessing in  the later steps and also to reduce the size of the generated data.

In [2]:
#import cv2
#import matplotlib.pyplot as plt
from keras.preprocessing import image
from tqdm import tqdm
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True
import keras
#define a function that reads a path to an image and returns a tensor suitable for keras


#Create Data Generator class that can generate data in batches and split them to train, test and validation sets
class DataGenerator(keras.utils.Sequence):
    
    #def __init__(self, list_file_paths, labels, batch_size=32, dim=(32,32,32), n_channels=1,
    #             n_classes=10, shuffle=True):
    def __init__(self, list_file_paths, labels, batch_size=32, shuffle=True):
        
        #Initialization
        #self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_file_paths = list_file_paths
        #self.n_channels = n_channels
        #self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __len__(self):
        #Denotes the number of batches per epoch
        return int(np.floor(len(self.list_file_paths) / self.batch_size))
    
    def __getitem__(self, index):
        #Generate one batch of data
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_file_paths[k] for k in indexes]

        # Generate data
        X = self.__data_generation(list_IDs_temp)
        y = self.labels[indexes]

        return X, y
    
    def on_epoch_end(self):
        #Updates indexes after each epoch
        self.indexes = np.arange(len(self.list_file_paths))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    def path_to_tensor(self, path):
        img = image.load_img(path, target_size=(256, 256))
        x = image.img_to_array(img)
        return np.expand_dims(x, axis=0)
    
    #define a function that reads a path to an image and returns a tensor suitable for keras
    #def paths_to_tensor(paths_list):
    #    img_list = [path_to_tensor(img_path) for img_path in tqdm(paths_list)]
    #    return np.vstack(img_list)
    
    def __data_generation(self, list_file_paths_temp):
        
        #Generates data containing batch_size samples
        # X : (n_samples, *dim, n_channels)
        # Initialization
        img_list = [self.path_to_tensor(img_path) for img_path in list_file_paths_temp]
        return np.vstack(img_list).astype('float32')/255



               

#normalize all images: divide the tensors by 255
#pre-process the data for Keras
#train_tensors = paths_to_tensor(images_train)#.astype('float32')/255
#valid_tensors = paths_to_tensor(images_val)#.astype('float32')/255
#test_tensors = paths_to_tensor(images_test)#.astype('float32')/255

print("atleast this is working!")



atleast this is working!


# Step 3 : Instantiate data generator objects

Here, we create the data generator objects for train. test and validation objects. 

In [3]:
# Parameters for the Data Generator
params = {'batch_size' : 64,
          'shuffle': True}

# creating generators
training_generator = DataGenerator(images_train, targets_train, **params)
validation_generator = DataGenerator(images_val, targets_val, **params)
testing_generator = DataGenerator(images_test, targets_test, **params)






# Step 4 : Construct & Train the Vanilla model

Here, we construct a Vanilla CNN model. It should contain a basic CNN layer, followed by RELU, Maxpool and Softmax layers respectively in that order. 

In [4]:

from  keras.layers  import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()

### TODO: Define your architecture.

model.add(Conv2D(filters = 32, kernel_size = 3, strides = 1, padding = 'same', activation = None, input_shape = (256,256,3)))
#model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
#model.add(Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same', activation = None))
#model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
#model.add(Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = 'same', activation = 'relu'))
#model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
#model.add(Conv2D(filters = 256, kernel_size = 3, strides = 2, padding = 'same', activation = 'relu'))
model.add(GlobalAveragePooling2D())

#model.add(Dropout(0.5))

model.add(Dense(10,activation = 'softmax'))

         

model.summary()






_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
global_average_pooling2d_1 ( (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 1,226
Trainable params: 1,226
Non-trainable params: 0
_________________________________________________________________


Compiling the model

In [5]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

We train the vanilla model here

In [7]:
from keras.callbacks import ModelCheckpoint  

### TODO: specify the number of epochs that you would like to use to train the model.

epochs = 25

### .

checkpointer = ModelCheckpoint(filepath='weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit_generator(generator=training_generator, validation_data=validation_generator, epochs= epochs, callbacks=[checkpointer], use_multiprocessing=True, workers=6)


Epoch 1/25
280/280 [==============================] - 174s 621ms/step - loss: 2.2995 - acc: 0.1164 - val_loss: 2.3014 - val_acc: 0.1013

Epoch 00001: val_loss improved from inf to 2.30143, saving model to weights.best.from_scratch.hdf5
Epoch 2/25
280/280 [==============================] - 171s 611ms/step - loss: 2.2986 - acc: 0.1152 - val_loss: 2.2972 - val_acc: 0.1187

Epoch 00002: val_loss improved from 2.30143 to 2.29722, saving model to weights.best.from_scratch.hdf5
Epoch 3/25
280/280 [==============================] - 171s 612ms/step - loss: 2.2978 - acc: 0.1148 - val_loss: 2.2971 - val_acc: 0.1152

Epoch 00003: val_loss improved from 2.29722 to 2.29712, saving model to weights.best.from_scratch.hdf5
Epoch 4/25
280/280 [==============================] - 172s 613ms/step - loss: 2.2957 - acc: 0.1179 - val_loss: 2.2975 - val_acc: 0.1134

Epoch 00004: val_loss did not improve
Epoch 5/25
280/280 [==============================] - 177s 633ms/step - loss: 2.2947 - acc: 0.1175 - val_loss

# Step 5 : Test the Vanilla Model

Here we test the vanilla model and get the accuracy of the model. We also calculate the multiclass logloss value in this step


# Step 6 : Load RESNET model

# Step 7 : Transfer Learning using the RESNET model

# Step 8 : Test the new model

# Step 9 : Show some image results with the new model

# Step 10 : Conclusion